In [1]:
import os
import json
import datetime
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [2]:
train_file_names = []

for root, dirnames, filenames in os.walk("./ARC-AGI-master/data/training"):
    for filename in filenames:
        if filename.endswith('.json'):
            train_file_names.append(filename)

In [21]:
bunch_of_series = []
i = 0

for name in train_file_names:
    with (open(f"./ARC-AGI-master/data/training/{name}") as temp_file):
        some_file = json.load(temp_file)
        
        try:
            prob_ex_3_placement_input = [[str(i[j]) for i in 
                                     some_file["train"][2]["input"]] for j in range
                                     (len(some_file["train"][2]["input"][0]))]
            prob_ex_3_placement_output = [[str(i[j]) for i in 
                                     some_file["train"][2]["output"]] for j in range
                                     (len(some_file["train"][2]["output"][0]))]
        #print(i)
        except IndexError:
            prob_ex_3_placement = {"input": "", "output": ""}
            
        try:
            prob_ex_4_placement_input = [[str(i[j]) for i in 
                                         some_file["train"][3]["input"]] for j in range
                                         (len(some_file["train"][3]["input"][0]))]
            prob_ex_4_placement_output = [[str(i[j]) for i in 
                                         some_file["train"][3]["output"]] for j in range
                                         (len(some_file["train"][3]["output"][0]))]
        except IndexError:
            prob_ex_4_placement = {"input": [""], "output": [""]}
        
        temp_series = pd.Series({
            "problem_id": name,
            "problem_example_1_input":  [[str(i[j]) for i in 
                                         some_file["train"][0]["input"]] for j in range
                                         (len(some_file["train"][0]["input"][0]))],
            "problem_example_1_output": [[str(i[j]) for i in 
                                         some_file["train"][0]["output"]] for j in range
                                         (len(some_file["train"][0]["output"][0]))],
            "problem_example_2_input":  [[str(i[j]) for i in 
                                         some_file["train"][1]["input"]] for j in range
                                         (len(some_file["train"][1]["input"][0]))],
            "problem_example_2_output": [[str(i[j]) for i in 
                                         some_file["train"][1]["output"]] for j in range
                                         (len(some_file["train"][1]["output"][0]))],
            "problem_example_3_input":  prob_ex_3_placement_input,
            "problem_example_3_output": prob_ex_3_placement_output,
            "problem_example_4_input":  prob_ex_4_placement_input, 
            "problem_example_4_output": prob_ex_4_placement_output,
            "problem_test_input":       [[str(i[j]) for i in 
                                         some_file["test"][0]["input"]] for j in range
                                         (len(some_file["test"][0]["input"][0]))],
            "problem_test_output":      [[str(i[j]) for i in 
                                         some_file["test"][0]["output"]] for j in range
                                         (len(some_file["test"][0]["output"][0]))],
        })
        
        i += 1
        bunch_of_series.append(temp_series)
        
arc_train = pd.concat(bunch_of_series, axis=1).T
arc_no_id = arc_train.loc[:, arc_train.columns != "problem_id"]
arc_no_id_hf = Dataset.from_pandas(arc_no_id)

In [45]:
with open(f"./ARC-AGI-master/data/training/{train_file_names[0]}") as f:
    some_file = json.load(f)
    print(some_file)

{'train': [{'input': [[0, 0, 5], [0, 5, 0], [5, 0, 0]], 'output': [[3, 3, 3], [4, 4, 4], [2, 2, 2]]}, {'input': [[0, 0, 5], [0, 0, 5], [0, 0, 5]], 'output': [[3, 3, 3], [3, 3, 3], [3, 3, 3]]}, {'input': [[5, 0, 0], [0, 5, 0], [5, 0, 0]], 'output': [[2, 2, 2], [4, 4, 4], [2, 2, 2]]}, {'input': [[0, 5, 0], [0, 0, 5], [0, 5, 0]], 'output': [[4, 4, 4], [3, 3, 3], [4, 4, 4]]}], 'test': [{'input': [[0, 0, 5], [5, 0, 0], [0, 5, 0]], 'output': [[3, 3, 3], [2, 2, 2], [4, 4, 4]]}]}


In [35]:
arc_no_id

,problem_example_1_input,problem_example_1_output,problem_example_2_input,problem_example_2_output,problem_example_3_input,problem_example_3_output,problem_example_4_input,problem_example_4_output,problem_test_input,problem_test_output
0,"[[0, 0, 5], [0, 5, 0], [5, 0, 0]]","[[3, 4, 2], [3, 4, 2], [3, 4, 2]]","[[0, 0, 0], [0, 0, 0], [5, 5, 5]]","[[3, 3, 3], [3, 3, 3], [3, 3, 3]]","[[5, 0, 5], [0, 5, 0], [0, 0, 0]]","[[2, 4, 2], [2, 4, 2], [2, 4, 2]]","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 5, 0], [0, 0, 5], [5, 0, 0]]","[[3, 2, 4], [3, 2, 4], [3, 2, 4]]"
1,"[[3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 8, 3], [3, 3, ...","[[2, 8, 1, 8, 2], [4, 3, 3, 3, 4], [1, 3, 3, 3...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 8, 1, 8, ...","[[8, 3, 8], [3, 1, 3], [8, 3, 8]]","[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], [4, 4, ...","[[1, 4, 7, 4, 1], [4, 4, 4, 4, 4], [7, 4, 4, 4...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,...","[[3, 8, 2, 1, 2, 8, 3], [8, 8, 8, 8, 8, 8, 8],..."
2,"[[0, 0, 2, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0,...","[[2, 1, 3, 4, 8, 6], [2, 1, 3, 4, 8, 6], [2, 1...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 5, ...","[[1, 1, 1], [2, 2, 2], [4, 4, 4]]","[[0, 2, 0, 0, 8, 0, 0, 4, 0, 0, 1, 0], [0, 2, ...","[[2, 8, 4, 1], [2, 8, 4, 1], [2, 8, 4, 1], [2,...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[2, 0, 0, 3, 0, 0, 8, 0, 0, 4, 0, 0, 6, 0, 0,...","[[2, 3, 8, 4, 6, 1, 7], [2, 3, 8, 4, 6, 1, 7],..."
3,"[[1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0...","[[1, 0, 0, 0, 0], [2, 0, 0, 1, 0], [1, 0, 0, 2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, ...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, ..."
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...,...,...,...
395,"[[1, 1, 3, 2], [1, 1, 3, 3], [3, 3, 1, 1], [2,...","[[1, 1, 3, 2], [1, 1, 3, 3], [3, 3, 1, 1], [2,...","[[4, 6, 6], [4, 4, 6], [4, 8, 8], [4, 6, 6], [...","[[4, 6, 6], [4, 4, 6], [4, 8, 8]]","[[2, 3, 4, 2, 3, 4], [3, 2, 4, 3, 2, 4]]","[[2, 3, 4], [3, 2, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ...","[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ...","[[5, 4, 6, 2, 5, 4, 6, 2], [4, 5, 6, 6, 4, 5, ...","[[5, 4, 6, 2], [4, 5, 6, 6], [5, 4, 4, 2]]"
396,"[[1, 1, 0], [1, 1, 0], [0, 1, 0]]","[[1, 1, 0, 0, 1, 1], [1, 1, 0, 0, 1, 1], [0, 1...","[[0, 1, 1], [0, 0, 1], [0, 1, 0]]","[[0, 1, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0], [0, 1...","[[0, 0, 0], [0, 0, 0], [0, 1, 1]]","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 1...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ...","[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ...","[[0, 0, 1], [0, 0, 0], [0, 1, 0]]","[[0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 1..."
397,"[[4, 4, 0], [4, 0, 0], [0, 0, 0], [3, 3, 0], [...","[[6, 6, 0], [6, 0, 0], [0, 0, 6]]","[[4, 4, 0], [0, 0, 0], [4, 0, 4], [3, 3, 3], [...","[[6, 6, 6], [6, 0, 0], [6, 0, 6]]","[[0, 0, 4], [0, 4, 4], [4, 4, 0], [0, 3, 0], [...","[[0, 6, 6], [6, 6, 6], [6, 6, 6]]","[[4, 0, 4], [4, 0, 0], [0, 0, 0], [3, 0, 0], [...","[[6, 0, 6], [6, 0, 0], [0, 6, 0]]","[[0, 4, 0], [4, 0, 4], [4, 0, 0], [3, 3, 3], [...","[[6, 6, 6], [6, 6, 6], [6, 0, 0]]"
398,"[[2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [2, 0, ...","[[2,

In [14]:
#tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", 
# trust_remote_code=True)
#model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", 
# trust_remote_code=True)

# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-vision-instruct", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-MoE-instruct",                                         trust_remote_code=True)

In [37]:
def grid2str(grid):
    return "\n".join("".join(str(i) for i in j) for j in grid)

num = 0

prompt = (
    f"Here are three examples of a puzzle — \n"
    f'Puzzle 1 input: \n\n{grid2str(arc_no_id_hf["problem_example_1_input"][num])} \n\n'
    f'Puzzle 1 output: \n\n{grid2str(arc_no_id_hf["problem_example_1_output"][num])} '
    f'\n\n\n'
    f'Puzzle 2 input: \n\n{grid2str(arc_no_id_hf["problem_example_2_input"][num])} \n\n'
    f'Puzzle 2 output: \n\n{grid2str(arc_no_id_hf["problem_example_2_output"][num])} '
    f'\n\n\n'
    f'Puzzle 3 input: \n\n{grid2str(arc_no_id_hf["problem_example_3_input"][num])} \n\n'
    f'Puzzle 3 output: \n\n{grid2str(arc_no_id_hf["problem_example_3_output"][num])} '
    f'\n\n\n'
    
    f"Hint: These puzzles can be imagined as grids and the digits represent different "
    f"colors. 0: black, 1: blue, 2: red, 3: green, 4: yellow, 5: silver, 6: pink, 7: "
    f"orange, 8: cyan, 9: brown. Each puzzle-solution pair has some transformation, for "
    f"instance, adding  a particular digit/color-block in-between two other "
    f"digits/color-blocks or splitting/combining the size of the overall grid. The "
    f"solution may not have anything to with the digits themselves or some numerical "
    f"computations. \n\n"
    
    f"Based on the examples, could you solve the following test puzzle? \n\n"
    f'{grid2str(arc_no_id_hf["problem_test_input"][num])} \n\n'
    f'Use chain of thought reasoning, try to identify the pattern, and provide only the '
    f'final answer to the test puzzle.'
)

print(prompt)
# prompt = f"Here's three examples of a puzzle \n"

# for i in range(4):
#    prompt += f'Puzzle {i + 1} input: \n\n {grid2str
#    (arc_no_id_hf["problem_example_{i}_input"][num])}'

Here are three examples of a puzzle — 
Puzzle 1 input: 

005
050
500 

Puzzle 1 output: 

342
342
342 


Puzzle 2 input: 

000
000
555 

Puzzle 2 output: 

333
333
333 


Puzzle 3 input: 

505
050
000 

Puzzle 3 output: 

242
242
242 


Hint: These puzzles can be imagined as grids and the digits represent different colors. 0: black, 1: blue, 2: red, 3: green, 4: yellow, 5: silver, 6: pink, 7: orange, 8: cyan, 9: brown. Each puzzle-solution pair has some transformation, for instance, adding  a particular digit/color-block in-between two other digits/color-blocks or splitting/combining the size of the overall grid. The solution may not have anything to with the digits themselves or some numerical computations. 

Based on the examples, could you solve the following test puzzle? 

050
005
500 

Use chain of thought reasoning, try to identify the pattern, and provide only the final answer to the test puzzle.


In [12]:
#if tokenizer.pad_token is None:
#    tokenizer.pad_token = tokenizer.eos_token
#
#def chat_with_model(prompt):
#    inputs = tokenizer.encode(prompt, return_tensors='pt').to("mps")
#    model.to("mps")
#    
#    output = model.generate(
#        inputs, 
#        max_new_tokens=300,
#        pad_token_id=tokenizer.pad_token_id
#    )
#    
#    response = tokenizer.decode(output[0], skip_special_tokens=True)
#    
#    return response #[len(prompt):].strip()

In [13]:
#response = chat_with_model(prompt)
#print(response)

In [40]:
print(grid2str(arc_no_id_hf["problem_test_output"][num]))

324
324
324


In [38]:
# from unittest.mock import patch
# from transformers.dynamic_module_utils import get_imports
# 
# device = "cpu"
# 
# def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
#     """Work around for https://huggingface.co/microsoft/phi-1_5/discussions/72."""
#     if not str(filename).endswith("/modeling_phimoe.py"):
#         return get_imports(filename)
#     imports = get_imports(filename)
#     imports.remove("flash_attn")
#     return imports

model_name = "microsoft/Phi-3.5-mini-instruct"

# create model
#with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports):
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    #device_map='auto' if torch.cuda.is_available() else "mps",
    trust_remote_code=True,
    attn_implementation="eager"
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

torch.random.manual_seed(0)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name, 
#     device_map="mps", 
#     torch_dtype="auto", 
#     trust_remote_code=True,
#     attn_implementation="eager"
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [44]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": f"{prompt}"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device="mps"
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    #"temperature": 0.9,
    #"do_sample": True,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

 They. learned.
. through. connection facilated.     [] connection facilated.     ]."".", and communication facilated.     ""."self".
. initi.

 ): The � []: The ibility.":".":"."""", self – and communication.flight.)) and communication.")); and self – communication –).).".".".""). "").""."".""."".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".“..".….“.“.“.“.“.“.“.“.“.“.“....`.`.```.`````.`.````.````````.`.`` "".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".".").".".".").".".".".").".".".")."".".").").".".").").".").".")).".(".")).".".(".")).".".".")).".".".")).".".".").".".".")).".")).".".".")).".(".")).".".".".".").").".").".").").".").").").").").").").").").").")).").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").").).").").").").").").").").").").").").").").").")